# `hv.Holomaps` with FLDAS dataset
Modified: Jun 2, 2019

Eastern & Southern Africa fldas data

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import numpy as np
import scipy as sp
import pandas as pd
import geopandas as gpd
import intake,param
    
from pathlib import Path
from pprint import pprint as pp
p = print 

from sklearn.externals import joblib
import pdb

from tqdm import tqdm, trange
import ipywidgets as iw

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr

from holoviews import opts
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.streams import Stream, param
from holoviews import streams
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts

import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf

hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'

In [ ]:
# Add the utils directory to the search path
UTILS_DIR = Path('../utils').absolute()
assert UTILS_DIR.exists()
if str(UTILS_DIR) not in sys.path:
    sys.path.insert(0, str(UTILS_DIR))
    print(f"Added {str(UTILS_DIR)} to sys.path")

pp(sys.path)
    

In [ ]:
from utils import get_mro as mro, nprint, dict2json, display_dict2json, is_valid_url
import utils

### Use holoview's colormaps module
<img src="../assets/hv_colormaps.jpg" alt="holoviews-colormaps" width="600"/>

In [ ]:
from holoviews.plotting import list_cmaps
# list_cmaps(provider='colorcet', category='Sequential')

In [ ]:
# Grab registered bokeh renderer
print("Currently available renderers: ", *hv.Store.renderers.keys())
renderer = hv.renderer('bokeh')

## Vector Tile overlay creation helpers
Modified: Jun 8, 2019

In [ ]:
from VectorTile import VectorTile

Quick example

In [ ]:
lat, lon = 48.8566101, 2.3514992
z = 15
vtile = VectorTile.from_latlon(lat, lon, z)
vtile.info()

gdf = vtile.to_gdf()
# display(gdf.head())

ndoverlay = vtile.to_ndoverlay()
# display(ndoverlay)

## Set default holoviews style options

In [ ]:
%opts Image [colorbar=True, tools=['hover']] Curve [tools=['hover']]

---
## Load Datasets

In [ ]:
# Southern Africa Dataset
fpath_sa = '/home/hayley/data/mint/FLDAS/FLDAS_NOAH01_A_SA_D.001/2019/04/FLDAS_NOAH01_A_SA_D.A201904*.001.nc'
fpath_ea = '/home/hayley/data/mint/FLDAS/FLDAS_NOAH01_A_EA_D.001/2019/04/FLDAS_NOAH01_A_EA_D.A201904*.001.nc'
ds_sa = xr.open_mfdataset(fpath_sa)
ds_sa = ds_sa.drop_dims('bnds')

ds_ea = xr.open_mfdataset(fpath_ea)
ds_ea = ds_ea.drop_dims('bnds')

         
# print(ds_ea)
# print(ds_sa)

In [ ]:
xrd_ea = ds_ea.persist()
xrd_sa = ds_sa.persist()

In [ ]:
# data variable list
varnames_ea = list(ds_ea.data_vars.keys())
varnames_sa = list(ds_sa.data_vars.keys())
varnames = varnames_ea
varname = varnames[3]
print(varname)

#set height, width for images (for plotting)
H_IMG, W_IMG = 400, 400

# create holoviews dataset containers 
kdims = ['X','Y','time']
hvd_ea = hv.Dataset(xrd_ea, kdims)
hvd_sa = hv.Dataset(xrd_sa, kdims)


In [ ]:
# Simple example of `hv.dim()` funnction to transform dimensions
# Useful to transform a dimension before mapping it to a visual attribute for plotting custromization
# src: http://holoviews.org/user_guide/Style_Mapping.html
hv.dim('X').apply(hvd_sa)
hv.dim('time').apply(hvd_sa)

In [ ]:
# basemap tile
basemap = gvts.EsriNatGeo
basemap *timg_ea

In [ ]:
# colormaps
## discretize it conveniently using holoview's "color_level" option
t_fixed = '2019-04-05'
varname = varnames[5] 
print("Selecting a datavariable at a fixed time point: ", t_fixed, varname)

# timg_ea = hvd_ea.select(time=t_fixed).to(gv.Image, kdims=['X', 'Y'], vdims=varname) #this returns a holomap, not a hv.Image object
# To construct an hv.Image object, we need to pass in the xr.DataArray (ie. one value variable)
print(xrd_ea[varname].isel(time=3) )
timg_ea = hv.Image(xrd_ea[varname].isel(time=3) , ['X','Y']) #Opt: vdims=varname
timg_sa = hv.Image(xrd_sa[varname].isel(time=3) , ['X','Y']) #Opt: vdims=varname
# print(timg_sa)
# timg_ea.opts(width=W_IMG, height=H_IMG)

In [ ]:
# Now let's see how we can use holoview's plotting.opts function with `cmap` and `color_level` keywords to 
# categorize the values to certain number of color_levels easily

( 
    timg_ea.relabel('default, continuous level cmapping').opts(cmap='PiYG') + \
    timg_ea.relabel('5 color levels').opts(cmap='PiYG', color_levels=5) + \
    timg_ea.relabel('11 color levels').opts(cmap='PiYG', color_levels=11)
).opts(width=300,height=300).cols(3)


In [ ]:
# create dmap 
xrd_ea_small = xrd_ea.isel(time=[3,4])
xrd_sa_small = xrd_sa.isel(time=[3,4])

## Wrong: this creates holomaps
# dmap_ea_small = hv.Dataset(xrd_ea_small, kdims).to(hv.Image, ['X','Y'], varname, label='EA')
# dmap_sa_small = hv.Dataset(xrd_sa_small, kdims).to(hv.Image, ['X','Y'], varname, label='SA')

# datashade opts
from colorcet import fire
dopts = dict(width=500, height=500,
            x_sampling=0.5, 
            y_sampling=0.5)

img_opts = dict(width=500, height=500,
                axiswise=True, framewise=False)


In [ ]:
def get_img(tidx, varname, region='EA'):
    xrd = xrd_ea if region == 'EA' else xrd_sa
    img = hv.Image(xrd.isel(time=tidx)[varname], ['X','Y'])    
    
#     datashade returns a hv.DynamicMap which dynamically re-renders this img as we zoom/pan
    return img.opts(**img_opts) #, cmap=fire, **dopts)

    # hover info
def get_hover_info(tidx, varname, region='EA'):
    xrd = xrd_ea if region == 'EA' else xrd_sa
    img = hv.Image(xrd.isel(time=tidx)[varname], ['X','Y'])
    return rasterize(img, width=10, height=10, streams=[streams.RangeXY])
                           

    

In [ ]:
agg_img = rasterize(timg_ea, width=100, height=100, streams=[streams.RangeXY])

In [ ]:
agg_dmap  = hv.DynamicMap(lambda tidx, varname: get_hover_info(tidx, varname, 'EA'),
                    kdims=['tidx', 'varname'])
# print(agg_dmap)
agg_dmap.redim.values(tidx=[0,1,2], varname=varnames[:2])

In [ ]:
def realize(t_vals, v_vals):
#     return datashade( 
#         dmap_ea.redim.range(tidx=(3,5)).redim.values(varname=varnames[0:3])
#     ) * agg_dmap.redim.values(tidx=t_vals, varname=v_vals)
    return agg_dmap.redim.values(tidx=t_vals, varname=v_vals)

In [ ]:
realize([0,1,2], varnames[::2])

In [ ]:
dmap_ea = hv.DynamicMap(lambda tidx, varname: get_img(tidx, varname, 'EA'),
                        kdims=['tidx', 'varname'])
dmap_varnames = dmap_ea.redim.range(tidx=(3,5))
temp = dmap_ea.redim.range(tidx=(3,5)).redim.values(varname=varnames[0:3])


In [ ]:
dmap_varnames.redim.values(varname=varnames[::2])


In [ ]:
temp = dmap_ea.redim.range(tidx=(3,5)).redim.values(varname=varnames[0:3])

dsed = datashade(temp) 
print(dsed[3,varnames[0]])
plot = renderer.get_plot(dsed)
print(plot)

In [ ]:
stream = streams.RangeXY(source=dsed)
rasterize(

In [ ]:
otemp = temp + temp.opts('RGB', width=300, height=300)
otemp.opts(
    opts.RGB(axiswise=True)
)

In [ ]:
%%opts RGB  {+axiswise -framewise}

hmap_ea = hv.HoloMap(
    {(tidx, varname): get_img(tidx, varname, 'EA') 
     for tidx in range(0,2) for varname in varnames[:2]},
    kdims=['tidx', 'varname']
)
hmap_ea.layout()

In [ ]:
@out_debug.capture(clear_output=True)
def show():
    display(
        hv.Curve([1,2,3])+\
        hmap_ea.grid()
    )
#         hmap_ea.opts(framewise=True).grid()
    
    
                        

In [ ]:
dmap_ea = hv.DynamicMap(lambda tidx, varname: get_img(tidx, varname, 'EA'), 
                        kdims=['tidx', 'varname'])    
dmap_sa = hv.DynamicMap(lambda tidx, varname: get_img(tidx, varname, 'SA'), 
                        kdims=['tidx', 'varname'])    

In [ ]:
dmap_ea.redim.values(tidx=[0,10,20,30],
                     varname=varnames[::8]
                    )

In [ ]:
print(dmap_ea_small)

In [ ]:
temp = datashade(dmap_ea_small)
print(temp)

Wow, this is really powerful. I love that I didn't need to categorize the values to 5 or 11 levels myself -> no new data variable for the result, so memory efficient too.

## cycle colormaps
Modified: Jun 7, 2019

In [ ]:
# gdf
# gdf = get_vectile_gdf
# lat, lon, z = -5, 37, 9
# print(utils.get_latlon('paris, france'))
z = 16
lat, lon = utils.get_latlon('paris, france')
      

In [ ]:
gdf = get_vectile_gdf_at_latlon(lat, lon, z)
print(len(gdf))
display(gdf.head())

In [ ]:
addr = input('Enter query address: ')
lat, lon = utils.get_latlon(addr)
print('lat, lon: ', lat, lon)
z = int(input('Enter zoom level [0,16]:'))
shapes = get_vectile_overlay_at_latlon(lat,lon, z)
shapes

# try:
# addr: national museum of scotland, edinburgh, scotland
# lat, lon:  55.9470714 -3.18933554234473


In [ ]:
shapes = get_vectile_overlay_at_latlon(lat,lon, z=16)


In [ ]:
shapes

In [ ]:
# hv.DynamicMap(get_vectile_overlay_at_latlon, kdims=['lat', 'lon', 'z']).
# redim.values(lat=[

## A list of Lat, Lon data for world cities
Modified: Jun 7, 2019
- Plot a table of word citiy location data
and link the table with a DynamicMap that shows the vector tiles at that location 
(with default zoom of 14(?) 


In [ ]:
latlons = pd.read_csv('/home/hayley/data/latlon/worldcities.csv')
latlons = latlons[['city_ascii', 'lat', 'lng', 'country']]

In [ ]:
sample_latlons = latlons.sample(frac=0.01)
len(sample_latlons)

In [ ]:
sample_latlons = (
    sample_latlons
    .rename({'city_ascii':'city'}, axis='columns')
    .sort_values(by='country')
#     .reset_index()
)

In [ ]:
latlon_table = hv.Table(sample_latlons, 
                        kdims = ['country', 'city', 'lat', 'lng'],
                        label='city_latlon')#.opts(selectable=True)

In [ ]:
latlon_table#.opts(tools=['tap'])

In [ ]:
df = pd.DataFrame(np.random.rand(100,3), columns=['1','2','3'])

table = hv.Table(data=df)
sel = hv.streams.Selection1D(source=table)
table

In [ ]:
sel1 = hv.streams.Selection1D(source=table)
sel2 = hv.streams.Selection1D(source=table)


In [ ]:
print(sel1.contents, sel2.contents)

In [ ]:
def get_vectile_from_index(table, index, z=14):
    """
    Args:
        - table (hv.Table)
        - index (int): table row index 
    Returns:
        - hv.ndoverlay of gv.shape
    """
    country, city, lat, lon = table.iloc[index].array().flatten()
    vectile = VectorTile.from_latlon(lat,lon,z)
    ndoverlay = vectile.to_ndoverlay()
    return ndoverlay.relabel(group=country, label=city)

def get_vectiles_from_inds(table, index, z=14):
    """
    index (list): a list of table row index
    Returns a hv.DynamicMap of vectile-ndoverlay objects (one ndoverlay for latlon location)
        - kdims of the dmap is ['country', 'city']
    """
    
    if not index: #no selection 
        return hv.DynamicMap({})
    return hv.DynamicMap( {(country, city): get_vectile_overlay_from_index(table, i, z)
                           for (country, city, _, _) in table.iloc[i].array().flatten()
                           for i in tqdm(index)},
                         kdims=['country', 'city'],
                        )

#     for i in tqdm(index):
#         lat, lon = latlon_table['lat'][i], latlon_table['lng'][i]
#         country, city = latlon_table['country'][i], latlon_table['city'][i]
#         overlay.append(get_vectile_overlay_at_latlon(lat,lon,z))
#     return hv.Overlay(overlay)

In [ ]:
stream = hv.streams.Selection1D(source=latlon_table)

In [ ]:
stream.contents

In [ ]:
get_vectile_from_index(latlon_table, 1)        
    

In [ ]:
out = iw.Output(layout={'border': '1px solid black'})
# out.capture(clear_output=True, wait=True)
# (
#     hv.DynamicMap(get_vectile_overlay_from_index, kdims='index')
#     .redim.range(index=(0,len(latlon_table)-1))
# ).opts(width=600, height=600)

In [ ]:
out


Modified: Jun 8, 2019

In [ ]:
@out.capture(clear_output=True, wait=True)
def test_get_vectile_overlay_from_index():
#     data = np.array([
#         ['Algeria', 'El Bayadh', 33.6904, 1.01],
#         ['Argentina', 'San Lorenzo', -28.12, -58.77],
#         ['Australia', 'Bicheno', -41.8784, 148.2886],
#         ['Australia', 'Carnarvon', -24.8998, 113.6501],
#         ['Azerbaijan', 'Agdam', 40.9053, 45.5564],
#         ['France', 'Paris', 48.8566101, 2.3514992]
#     ], dtype=object) #dtype=object is important. It prevents the floats to be casted as a string
# #     table = hv.Table(data, kdims = ['country', 'city', 'lat', 'lon'])  
    
    table=latlon_table
    vectile = get_vectile_overlay_from_index(table, 28, z=15)

    display((basemap*vectile).opts(width=500, height=500) + table)
test_get_vectile_overlay_from_index()

WOOHOO!!!!!! IT WORKS!

In [ ]:
%%opts Shape (alpha=1)
@out.capture(clear_output=True, wait=True)
def test_get_vectile_overlay_from_index():
    data = np.array([
        ['Algeria', 'El Bayadh', 33.6904, 1.01],
        ['Argentina', 'San Lorenzo', -28.12, -58.77],
        ['Australia', 'Bicheno', -41.8784, 148.2886],
        ['Australia', 'Carnarvon', -24.8998, 113.6501],
        ['Azerbaijan', 'Agdam', 40.9053, 45.5564],
        ['France', 'Paris', 48.8566101, 2.3514992],
        ['South Korea', 'Jeonju', 35.8242715, 127.1480657 ],
        ['United States', 'USC', 4.0254818, -118.284388537815],
        ['China', 'Shanghai', 1.2252985, 121.4890497],
        ['United States', 'Kansas', 39.115386,-94.608631]
    ], dtype=object) #dtype=object is important. It prevents the floats to be casted as a string
    table = hv.Table(data, kdims = ['country', 'city', 'lat', 'lon'])    
    vectile = get_vectile_from_index(table, -1, z=12)
#     display( (basemap * vectile).opts(width=500, height=500) + table)
    display( vectile.opts(width=500, height=500) + table)

    
test_get_vectile_overlay_from_index()

## If the Selection1D works...
Modified: Jun 8, 2019

In [ ]:
gv.Tiles(url, name="VectorTile")


In [ ]:
        self.tile_url = f'https://tile.nextzen.org/tilezen/vector/v1/{self.size}/{self.layer}/{self.z}/{self.xtile}/{self.ytile}.{self.fformat}?api_key=GpjLSbvrQsa98MgMMuodzw'


In [ ]:
url = 'https://tile.nextzen.org/tilezen/vector/v1/256/all/{Z}/{X}/{Y}.json?api_key=GpjLSbvrQsa98MgMMuodzw'

In [ ]:
gv.Tiles(

In [ ]:
base

## Panel Dashboards
Modified: Jun 10, 2019


In [ ]:
# import param
import panel as pn
# from holoviews.streams import Params
from geoviews.tile_sources import EsriImagery
from colorcet import palette

In [ ]:
#dataset: hvd_ea, hvd_sa
xr_datasets = {
    'EA': xrd_ea,
    'SA': xrd_sa
}

hv_datasets = {
    'EA': hvd_ea,
    'SA': hvd_sa
}
april = pd.date_range('2019-04-01', periods=30)

# options
## datashade option


In [ ]:

out_debug = iw.Output(layout={'border': '1px solid black'})
out_debug

In [ ]:
@out_debug.capture(clear_output=True)
class FLDASExplorer(param.Parameterized):
    region = param.ObjectSelector(default =  'Southern Africa',
                                  objects=['Eastern Africa', 'Southern Africa', 'all'])
    variable = param.ObjectSelector(default='SoilMoi00_10cm_tavg', 
                                    objects=varnames)
    cmap = param.ObjectSelector('fire', objects=['fire','bgy','bgyw','bmy','gray','kbc'])
    
    dopts = dict(
        x_sampling=0.5,
        y_sampling=0.5,
        width=500,
        height=500)
    
    iopts = dict()
    
#     @param.depends('region', 'variable')
#     def load_data(self):
    def make_view(self, **kwargs):
        print(self.region, self.variable)

        hvd = hvd_ea if self.region == 'Eastern Africa' else hvd_sa
        img_dmap = hvd.to(hv.Image, ['X','Y'], self.variable)
        

        return datashade(img_dmap,
                         cmap=self.cmap,
                         **dopts)
        

explorer = FLDASExplorer()
dmap = explorer.make_view()
display(pn.Row(explorer.param, dmap))

In [ ]:
april = pd.date_range('2019-04-01', periods=30)

datashade(hvd_ea.to(hv.Image, ['X','Y'], varname))

In [ ]:
hv.Image.range

## FLDASExplorer with panel
Modified: Jun 13, 2019

In [ ]:
import datetime as dt

class FLDASExplorer(param.Parameterized):
    region = param.ObjectSelector(default='EA', objects=['EA', 'SA'])
    varname = param.ObjectSelector(default=varnames[0],objects=varnames)
    time = param.Date(dt.datetime(2019,4,1), bounds=(dt.datetime(2019, 4, 1), dt.datetime(2019, 4, 30)))
    
    @param.depends('region', 'varname', 'time')
    def view(self):
        xrd = xrd_ea if self.region == 'EA' else xrd_sa
        img = hv.Image(xrd.sel(time=self.time)[self.varname], ['X','Y'])    

    #     datashade returns a hv.DynamicMap which dynamically re-renders this img as we zoom/pan
        return datashade(img.opts(**img_opts), cmap=fire, **dopts)


In [ ]:
explorer = FLDASExplorer()
# img_dmap = hv.DynamicMap(explorer.view)
app = pn.Row( explorer.param, explorer.view)

In [ ]:
app.servable()

### Generate a varname dropdown box
Modified: Jun 3, 2019

In [ ]:
%%opts Image [width=600, height=600]
var_tmap = gv.Dataset(xrd_ea, kdims=kdims, vdims=varname, crs=ccrs.PlateCarree()).to(gv.Image, kdims=['X','Y'], vdims=varname)
var_tmap.redim(X='Lon', Y='Lat')

In [ ]:
def get_var_tmap(xr_dataset, varname, label=None, cmap=None):
    if label is None:
        label = varname
    tmap = gv.Dataset(xr_dataset, kdims=kdims, vdims=varname).to(gv.Image, kdims=['X','Y'], vdims=varname, label=label).redim(X='Lon', Y='Lat')
    print('type of tmap before datashade: ', type(tmap))
    print('\t\t after datashade: ', type(tmap))
    tmap = datashade(tmap, cmap=cmap)
    return tmap

def get_var_timg(xr_dataset, varname, t_idx, label=None, cmap=None):
    if label is None:
        label = varname  
    timg = gv.Dataset(xr_dataset.isel(time=t_idx), kdims=['X','Y'], vdims=varname).to(gv.Image, kdims=['X','Y'], vdims=varname, label=label).redim(X='Lon', Y='Lat')
    print('type of timg before datashade: ', type(timg))
    print('\t\t after datashade: ', type(timg))
    timg = datashade(timg, cmap=cmap)
    return timg


def test_get_timg():
    varname = varnames[1]
    t_idx = 1
    from matplotlib.cm import viridis

    timg = get_var_timg(xrd_ea, varname, t_idx, cmap=viridis)
    
    print('varname: ', varname, 't_idx: ', t_idx)
    display(timg)
    

In [ ]:
%%opts Image [width=600, height=600, tools=['hover']]
test_get_timg()

Let's add a base landscape vector features


In [ ]:
# %%opts Image [global_extent=True, height=600, width=600] (alpha=0.7) Image.EA (cmap='kbc') Image.SA (cmap='fire')
sq_cmaps = list_cmaps(provider='colorcet', category='Sequential')
varname = varnames[1]
t_idx = 1
timg_ea = get_var_timg(xrd_ea, varname, t_idx, label='EA', cmap=c_cm.kbc)
timg_sa = get_var_timg(xrd_sa, varname, t_idx, label='SA', cmap=c_cm.fire)



In [ ]:
overlay = timg_ea + timg_sa
print(overlay)

In [ ]:
# Let's see the current plot.state of timg_ea andn timg_sa
plot_ea = renderer.get_plot(timg_ea)
plot_sa = renderer.get_plot(timg_sa)

In [ ]:
def print_plot_state(hvElem, renderer):
    plot = renderer.get_plot(hvElem)
    print("Plot state: ")
    plot.print_param_values()
    
def print_param_state(hvElem):
    print("param values: ")
    hvElem.print_param_values()
    
def print_all_states(hvElem, renderer):
    print("="*80)
    print_param_state(hvElem)
    
    print("="*80)
    print_plot_state(hvElem, renderer)
    
def test_print_param_state():
    print_param_state(timg_sa)
def test_print_plot_state():
    print_plot_state( timg_sa, renderer)
def test_print_all_states():
    print_all_states(timg_sa, renderer)   
    
# test_print_param_state()
# test_print_plot_state()
# test_print_all_states()


In [ ]:
tmap_ea = get_var_tmap(xrd_ea, varname, label='EA', cmap=c_cm.kbc)
tmap_sa = get_var_tmap(xrd_sa, varname, label='SA', cmap=c_cm.fire)

In [ ]:
%%opts RGB [global_extent=False, height=500, width=500, tools=['hover']] (alpha=0.7) 
temp = (tmap_ea + tmap_sa).opts(width=500, height=500)
temp


In [ ]:
# Get tiles
tile = gvts.EsriImagery

In [ ]:
%%opts RGB [global_extent=False, height=H_IMG, width=W_IMG, tools=['hover']] (alpha=0.7) 
tile * tmap_ea * tmap_sa * gf.coastline

---
## TODO: 
Modified: Jun 4, 2019
- [ ] add latlon PointXY stream -> plot across time for this particular variable
- [ ] datashader statistics on a selected region by adding an extra layer on top of this:)
- [ ] Work with osm and spacenet dataset

In [ ]:
lon,lat = (22.35, -11.45)
pt = gv.Points([ (lon,lat) ], ['X', 'Y'])
LonLat = Stream.define('LonLat', X=22.35, Y=-11.45)

pt_stream = LonLat()
print(pt_stream.X, pt_stream.Y)

In [ ]:
tmap_ea * pt.opts(size=10, color='r')

In [ ]:
# PointerXY, a Linked Stream, to get a 2D dataview across Lat or Lon at the specific location and time
# We can explicitly add a source to the stream by passing the source object (eg. holompas, image) as an 
# argument to 'source' at the construction time of the Linked Stream object
pointer = streams.PointerXY(source=tmap_ea)
print(pointer.source is tmap_ea)

In [ ]:
# Let's see if it works
hv.DynamicMap(lambda x,y: hv.Points([(x,y)]).opts(size=10, framewise=True), streams=[pointer])

Awesome!


We can add the pointer stream to its original plot source (ie. `tmap_ea`)
def tmap_and_pointer(

In [ ]:
tmap_ea.redim.values(time=pd.date_range('2019-04-13', '2019-04-20'))

In [ ]:
hv.Dataset(xrd_ea.sel(X=lon, Y=lat, method='nearest'), 'time')

In [ ]:
def get_tseries_at_lon_lat(lon, lat):
    return (
        hv.Curve(xrd_ea.sel(X=lon, Y=lat, method='nearest'), 'time', varname)
        .relabel(f'Lon: {lon}, Lat: {lat}')
        .opts(width=1000, framewise=True)
    )


In [ ]:
def test_get_tseries_at_lon_lat():
    lon, lat = 35, -5
    display(get_tseries_at_lon_lat(lon, lat))
test_get_tseries_at_lon_lat()

In [ ]:
dmap = hv.DynamicMap(get_tseries_at_lon_lat, streams = [pointer.rename(x='lon', y='lat')])

In [ ]:
dmap.streams[0].source is pointer.source


In [ ]:
tmap_sa

In [ ]:
tmap_ea + dmap.opts(xticks=30)

The timeseries at a specific lat,lon is working! nice:)

## Basemap + EA, SA variable data + timeseries at lon/lat 
Modified: Jun 6, 2019

Let's put these together. Then we will 
- [ ] add another interactivity to select an area on the timage map and compute some meaning statistics on that region 
    - [ ] Use datashader's functionality here?
    
- [ ] take a look at the osm vector tile data (on the 2D space) 
- [ ] compute infrastructure distribution of 
    - [ ] the entire area
    - [ ] selected area
    
---
## Extra todos:
Modified: Jun 7, 2019
- [ ] if any of the coordinate of the pointer stream is out of bound, all tseries values should be zeros

 

1. Putting things together

In [ ]:
(
    (tile * tmap_ea).opts(width=1000,height=700) + \
    dmap.opts(xticks=30)
).cols(1)

In [ ]:
def get_tseries(xrd, varname, lon,lat):
    tseries = xrd[varname].sel(X=lon, Y=lat, method='nearest')
    return gv.Dataset(tseries, 'time', varname).to(hv.Curve)


In [ ]:
renamed_stream = LonLat().rename(X='lon', Y='lat')
# dmap_tseries = hv.DynamicMap(get_tseries, streams=[pt_stream.rename(X='lon', Y='lat')])
dmap_tseries = hv.DynamicMap(lambda lon,lat: get_tseries(xrd_ea, varnames[1], lon, lat), 
                            streams=[renamed_stream])

In [ ]:
dmap_tseries.opts(width=800, height=500)

In [ ]:
lons, lats, times = map(np.array, ds_ea[varname].coords.values())

In [ ]:

for i,(lon,lat) in enumerate(zip(lons[::10],lats[::10])):
    if i>9:
        break
    print(i, "lon,lat: ", lon,lat)
    dmap_tseries.event(lon=lon, lat=lat)
    time.sleep(3)
    
        

In [ ]:
dmap_tseries

In [ ]:

for i,(lon,lat) in enumerate(zip(lons[::10],lats[::10])):
    if i>5:
        break
    print(i, "lon,lat: ", lon,lat)
    dmap_tseries.event(lon=lon, lat=lat)
    time.sleep(1)
    

In [ ]:
dmap_tseries.opts(width=1000,height=300,framewise=True)

---
## Vector Tile Sources
Modified: Jun 5, 2019

In [ ]:
tile_url ='https://tile.nextzen.org/tilezen/vector/v1/512/all/16/19293/24641.json?api_key=GpjLSbvrQsa98MgMMuodzw'
gv.Tiles(tile_url, name="vector_tile").opts(width=600, height=550)


In [ ]:
import requests

In [ ]:
%%time
r = requests.get(url=tile_url)

In [ ]:
data = r.json()

In [ ]:
import json
with open('./temp.geojson', 'w') as f:
    json.dump(data,f)

In [ ]:
gdf2 = gpd.read_file('./temp.geojson')
gdf2.head()

In [ ]:
test_get_vectile_overlay_at_latlon()

In [ ]:
lat, lon = (-10, 30)
z=16
display(get_vectile_overlay_at_latlon(lat, lon, z))

In [ ]:
VectorTile.deg2tile_xy(lat,lon,z)

In [ ]:
VectorTile.tile_xyz2deg(19293, 24641, 16)

In [ ]:
vectile = hv.NdOverlay({i:gv.Shape(geom) for i,geom in enumerate(gdf2.geometry)})

In [ ]:
vectile

In [ ]:
f

Let's get a better vector features with national boundaries. `geoviews.features` is a warpper of `cartopy.crs.feature` which supports easy feature collection downloading from `Natural Earth Data`.
    - [link](https://is.gd/jmmPRV)


In [ ]:
gf_land_110 = gv.Feature(cf.NaturalEarthFeature('cultural', 'admin_0_boundary_lines_land', '110m'))

# nice bboxfinder
# https://is.gd/L163zr
extents = (-32.343750,-43.580391,69.257813,47.279229)
xlim = (extents[0], extents[2])
ylim = (extents[1], extents[3])

# reset the extent for visualization only
# gf_land_110.extents =  extents
# gf_land_110

In [ ]:
timg_ea.bounds

In [ ]:
gf_land_110.extents

In [ ]:
%%opts Feature [global_extent=False]
gf_land_110

In [ ]:
%%opts WMTS [global_extent=True, height=500, width=500] Image [height=500, width=500] (alpha=0.7) Image.EA (cmap='kbc') Image.SA (cmap='fire')
# gvts.OSM * timg_ea
gvts.EsriNatGeo * timg_ea
# timg_ea 



In [ ]:
gvts.OSM

In [ ]:
%%opts  Image [ height=600, width=600] (alpha=0.7) Image.EA (cmap='kbc') Image.SA (cmap='fire')

# gf_land_110 * timg_ea

In [ ]:
gf_land_110.extents=tuple([None for i in range(4)])

In [ ]:
gf_land_110

In [ ]:
timg_ea.opts(opts.Image(xlim=xlim, ylim=ylim))

In [ ]:
(timg_ea * timg_sa * gf_land_110)


In [ ]:


# ( 
#     (timg_ea * gf_land_110) + \
#     (timg_sa * gf_land_110) + \
#     (timg_ea * timg_sa * gf_land_110)
# ).cols(1)

In [ ]:
%%opts Feature [global_extent=True, height=600, width=1200] Points [global_extent=True] (size=10)
# WMTS [projection=ccrs.Geostationary()] Image [projection=ccrs.Geostationary()]
lon, lat = 40.0, 8.0
pt = gv.Points([(lon,lat)], crs=ccrs.PlateCarree())
(gf.land * pt)

---
## Load vector data using geopandas interface
Modified: Jun 4, 2019

In [ ]:
fpath = '../assets/cities.csv'
gdf = gpd.read_file(fpath)
gdf.head()

---
## Load dataset using intake

In [ ]:
cat = intake.open_catalog('./fldas_catalog.yml')
list(cat)

In [ ]:
ds_ea1 = cat.fldas_noah01_a_regional_daily_single(region='EA', day=2)
xr_ea1 = ds_ea1.to_dask()

In [ ]:
xr_ea1

In [ ]:
ds_ea = cat.fldas_noah01_a_regional_daily_all(region='EA')
ds_sa = cat.fldas_noah01_a_regional_daily_all(region='SA')

In [ ]:
%%time
ea = ds_ea.read()

In [ ]:
%%time
sa = ds_sa.read()

In [ ]:
ea = ea.drop_dims('bnds')
sa = sa.drop_dims('bnds')

In [ ]:
ea


In [ ]:
kdims = ['X', 'Y', 'time']
varnames = list(ea.data_vars.keys())
print(varnames)



In [ ]:
hvd_ea = hv.Dataset(ea, kdims=kdims)
hvd_sa = hv.Dataset(sa, kdims=kdims)
# print(hvd_ea)

In [ ]:
def get_hvm_var(xr_dset, varname):
    
    hvd_var = hv.Dataset(xr_dset[varname], kdims=kdims)
    hvd_var.to(gv.Image, ['X','Y'], dynamic=True)
    
get_hvm_var(hvd_sa, varnames[2])

In [ ]:
def get_varimg(xrd, varname, t):
    hvd = hv.Dataset(xrd[varname].isel(time=t), kdims=['X','Y'])
    return hvd.to(gv.Image, kdims=['X','Y'], vdims=varname)

varname = varnames[3]
t = 19
img0 = get_varimg(ea, varname, t)

In [ ]:
dmap = hv.DynamicMap(lambda varname, t: get_varimg(ea, varname, t),
            kdims=['varname', 't'])
              

In [ ]:
dmap[varnames[-1,1]

In [ ]:
varname = varnames[1]
hv.Dataset(ea[varname], kdims=['X','Y','time']).to(gv.Image, ['X','Y'], dynamic=True)

## Playground
Modified: Jun 13, 2019

In [ ]:
timg_ea

In [ ]:
tile = gv.tile_sources.EsriImagery
tile

In [ ]:
%%opts Overlay [global_extent=True, width=600, height=600]
gf.coastline * img0

In [ ]:
img0